In [26]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split


In [27]:
stock_data = yf.download('AAPL', start='2015-01-01', end='2023-01-01', interval='1d')
stock_data = stock_data[['Close']]
stock_data.dropna(inplace=True)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data)


[*********************100%***********************]  1 of 1 completed


In [28]:
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data to be 3D for LSTM (samples, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Step 4: Build the LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 5: Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - loss: 0.0626 - val_loss: 9.7239e-04
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - loss: 0.0033 - val_loss: 7.0141e-04
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 0.0025 - val_loss: 6.5740e-04
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0027 - val_loss: 6.9501e-04
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - loss: 0.0019 - val_loss: 6.1664e-04
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0019 - val_loss: 5.4754e-04
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - loss: 0.0021 - val_loss: 5.4622e-04
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - loss: 0.0023 - val_loss: 6.2257e-04
Epoch 9/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 0.0023 - val_loss: 7.3151e-04
Epoch 10/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0021 - val_loss: 8.6005e-04
Epoch 11/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 0.0021 - val_loss: 8.2424e-04
Epoch 12

Model Selection

In [30]:


# Step 6: Predictions and Evaluation
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Invert scaling for meaningful predictions
y_test = scaler.inverse_transform([y_test])

# Calculate performance metrics
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test[0], predictions[:, 0])
print("Mean Squared Error:", mse)



13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
Mean Squared Error: 6.733902618142784
